# Laboratoire 4 _ CEG 4536

In [2]:
!nvidia-smi

Tue Dec  3 21:02:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Tache 1 - Mémoire partagée



In [3]:
%%writefile Tache1.cu
#include <cuda_runtime.h>
#include <iostream>

// Taille du bloc
#define BLOCK_SIZE 32

// Kernel de transposition avec mémoire partagée et padding
__global__ void transposeWithSharedMemory(float *input, float *output, int width, int height) {
    __shared__ float tile[BLOCK_SIZE][BLOCK_SIZE + 1]; // +1 pour éviter les conflits de banque

    int x = blockIdx.x * BLOCK_SIZE + threadIdx.x;
    int y = blockIdx.y * BLOCK_SIZE + threadIdx.y;

    if (x < width && y < height) {
        tile[threadIdx.y][threadIdx.x] = input[y * width + x];
    }

    __syncthreads();

    int transposedX = blockIdx.y * BLOCK_SIZE + threadIdx.x;
    int transposedY = blockIdx.x * BLOCK_SIZE + threadIdx.y;

    if (transposedX < height && transposedY < width) {
        output[transposedY * height + transposedX] = tile[threadIdx.x][threadIdx.y];
    }
}

int main() {
    // Dimensions de la matrice
    int width = 1024;
    int height = 1024;

    size_t size = width * height * sizeof(float);

    // Allocation de mémoire hôte
    float *h_input = (float *)malloc(size);
    float *h_output = (float *)malloc(size);

    // Initialisation de la matrice
    for (int i = 0; i < width * height; i++) {
        h_input[i] = static_cast<float>(i);
    }

    // Allocation de mémoire GPU
    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);

    // Copie des données vers le GPU
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    // Configuration des dimensions du kernel
    dim3 blockDim(BLOCK_SIZE, BLOCK_SIZE);
    dim3 gridDim((width + BLOCK_SIZE - 1) / BLOCK_SIZE, (height + BLOCK_SIZE - 1) / BLOCK_SIZE);

    // Lancement du kernel
    transposeWithSharedMemory<<<gridDim, blockDim>>>(d_input, d_output, width, height);

    // Copie des résultats vers l'hôte
    cudaMemcpy(h_output, d_output, size, cudaMemcpyDeviceToHost);

    // Nettoyage et liberation de la mamoire
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);

    return 0;
}


Writing Tache1.cu


In [4]:
!nvcc Tache1.cu -o Tache1

In [5]:
!./Tache1

In [6]:
!nvprof ./Tache1

==8020== NVPROF is profiling process 8020, command: ./Tache1
==8020== Profiling application: ./Tache1
==8020== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   66.31%  1.6614ms         1  1.6614ms  1.6614ms  1.6614ms  [CUDA memcpy DtoH]
                   31.35%  785.44us         1  785.44us  785.44us  785.44us  [CUDA memcpy HtoD]
                    2.34%  58.688us         1  58.688us  58.688us  58.688us  transposeWithSharedMemory(float*, float*, int, int)
      API calls:   97.53%  189.43ms         2  94.713ms  74.266us  189.35ms  cudaMalloc
                    2.09%  4.0564ms         2  2.0282ms  950.42us  3.1060ms  cudaMemcpy
                    0.16%  307.07us         2  153.54us  103.80us  203.28us  cudaFree
                    0.11%  204.58us         1  204.58us  204.58us  204.58us  cudaLaunchKernel
                    0.10%  195.52us       114  1.7150us     151ns  71.307us  cuDeviceGetAttribute
              

# Resumé et explication

Le profilage du kernel montre que les copies de données entre l’hôte et le dispositif GPU dominent le temps d’exécution total, représentant 66,31% (DtoH) et 31,35% (HtoD) du temps GPU. Cela indique que l’efficacité globale est limitée par la latence de transfert mémoire, ce qui est typique pour des opérations impliquant de grandes quantités de données. Le kernel transposeWithSharedMemory, bien qu’optimisé avec de la mémoire partagée et un padding pour éviter les conflits de banque, ne représente que 2,34% du temps total GPU. Cela suggère que les optimisations sur les accès mémoire dans le kernel sont efficaces, mais leur impact est masqué par les coûts de transfert.

Le temps API est majoritairement consacré à cudaMalloc (97,53%), ce qui est attendu pour l'allocation initiale de mémoire GPU. Les temps associés à cudaMemcpy et cudaFree sont faibles par rapport aux transferts eux-mêmes, montrant une gestion mémoire correcte. L'ajout de padding (+1 dans la mémoire partagée) semble efficace pour éviter les conflits de banque, mais des améliorations peuvent encore être explorées, comme la réduction des transferts mémoire en regroupant plusieurs opérations dans une seule passe ou en utilisant des outils avancés pour des schémas d'accès encore plus efficaces.



------------------------------------------------------------------



-----------------------------------------------------------------------------


# Tache 2 - Optimisation du Kernel

Reduction parallèle avec mémoire partagée et shuffle

In [8]:
%%writefile Tache2.cu
#include <cuda_runtime.h>
#include <iostream>

#define BLOCK_SIZE 256

__global__ void reduceOptimized(const float *input, float *output, int size) {
    __shared__ float sharedData[BLOCK_SIZE];

    int tid = threadIdx.x;
    int globalIdx = blockIdx.x * blockDim.x * 2 + threadIdx.x;

    float sum = 0.0f;
    if (globalIdx < size) {
        sum += input[globalIdx];
        if (globalIdx + blockDim.x < size) {
            sum += input[globalIdx + blockDim.x];
        }
    }
    sharedData[tid] = sum;

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 32; stride /= 2) {
        if (tid < stride) {
            sharedData[tid] += sharedData[tid + stride];
        }
        __syncthreads();
    }

    if (tid < 32) {
        sum = sharedData[tid];
        #pragma unroll
        for (int offset = 16; offset > 0; offset /= 2) {
            sum += __shfl_down_sync(0xFFFFFFFF, sum, offset);
        }
        if (tid == 0) {
            sharedData[0] = sum;
        }
    }

    if (tid == 0) {
        output[blockIdx.x] = sharedData[0];
    }
}

int main() {
    int size = 1 << 20;  // 1 million d'éléments
    size_t bytes = size * sizeof(float);

    float *h_input = (float *)malloc(bytes);
    float *h_output;

    // Initialiser les données
    for (int i = 0; i < size; i++) {
        h_input[i] = 1.0f;
    }

    float *d_input, *d_output;
    cudaMalloc(&d_input, bytes);

    int threads = BLOCK_SIZE;
    int blocks = (size + threads * 2 - 1) / (threads * 2);

    h_output = (float *)malloc(blocks * sizeof(float));
    cudaMalloc(&d_output, blocks * sizeof(float));

    cudaMemcpy(d_input, h_input, bytes, cudaMemcpyHostToDevice);

    reduceOptimized<<<blocks, threads>>>(d_input, d_output, size);
    cudaDeviceSynchronize();

    cudaMemcpy(h_output, d_output, blocks * sizeof(float), cudaMemcpyDeviceToHost);

    float finalResult = 0.0f;
    for (int i = 0; i < blocks; i++) {
        finalResult += h_output[i];
    }

    std::cout << "Résultat final : " << finalResult << std::endl;

    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);

    return 0;
}


Writing Tache2.cu


In [9]:
!nvcc -o Tache2 Tache2.cu

In [10]:
!./Tache2

Résultat final : 524288


In [11]:
!nvprof ./Tache2

==9713== NVPROF is profiling process 9713, command: ./Tache2
Résultat final : 524288
==9713== Profiling application: ./Tache2
==9713== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   95.20%  765.76us         1  765.76us  765.76us  765.76us  [CUDA memcpy HtoD]
                    4.48%  36.064us         1  36.064us  36.064us  36.064us  reduceOptimized(float const *, float*, int)
                    0.31%  2.5280us         1  2.5280us  2.5280us  2.5280us  [CUDA memcpy DtoH]
      API calls:   99.11%  191.53ms         2  95.764ms  77.585us  191.45ms  cudaMalloc
                    0.49%  952.33us         2  476.17us  27.288us  925.04us  cudaMemcpy
                    0.17%  335.42us         2  167.71us  134.59us  200.83us  cudaFree
                    0.11%  208.05us         1  208.05us  208.05us  208.05us  cudaLaunchKernel
                    0.09%  167.80us       114  1.4710us     137ns  83.408us  cuDeviceGetAttribut

# Resumé et explication

Le kernel optimisé de réduction parallèle montre une efficacité notable avec seulement 36,064 µs consacrées à son exécution, représentant 4,48% du temps GPU. La majeure partie du temps GPU est cependant consacrée aux transferts de données entre l'hôte et le dispositif, avec 95,20% (765,76 µs) pour la copie HtoD et 0,31% (2,528 µs) pour la copie DtoH. Cela met en évidence que, comme souvent dans les applications CUDA, le transfert mémoire est un goulot d'étranglement.

L'utilisation de la mémoire partagée et des instructions **__shfl_down_sync** permet une réduction efficace des données, exploitant la parallélisation intra-warp pour minimiser les étapes de synchronisation nécessaires. Le résultat final attendu (524288) confirme la précision de l'algorithme. Les appels API dominent également le temps total, en particulier cudaMalloc (99,11%), ce qui est typique pour l’allocation initiale de la mémoire GPU.

L'optimisation des transferts mémoire, par exemple en regroupant davantage de calculs sur GPU avant de renvoyer les données, pourrait améliorer encore les performances globales de cette implémentation. Le kernel en lui-même est bien optimisé, mais les coûts de transfert masquent une partie de son efficacité.